In [1]:
import numpy as np
import dolfin as dl
from time import time
from nalger_helper_functions import pointwise_observation_matrix, points_inside_mesh, dofs_that_contribute_to_function_at_points

In [2]:
mesh = dl.UnitSquareMesh(351,363)
V = dl.FunctionSpace(mesh, 'CG', 2)
u = dl.Function(V)
u.vector()[:] = np.random.randn(V.dim())

N = 400
d = mesh.geometric_dimension()
pp = np.random.randn(N, d)

inside_mesh_mask = points_inside_mesh(pp, mesh) # <-- True where point is in mesh, false where point is outside
outside_mesh_mask = np.logical_not(inside_mesh_mask)
B = pointwise_observation_matrix(pp, V) # <-- makes scipy CSR matrix

In [3]:
x1 = np.zeros(N)
for k in range(N):
    if np.all(np.zeros(2) <= pp[k,:]) and np.all(pp[k,:] <= np.ones(2)):
        x1[k] = u(dl.Point(pp[k,:]))

x2 = np.zeros(N)
x2 = B * u.vector()[:]

err_pointwise_observation_matrix = np.linalg.norm(x2 - x1)
print('err_pointwise_observation_matrix=', err_pointwise_observation_matrix)

err_pointwise_observation_matrix= 8.460899244930976e-16


In [4]:
exterior_fill_value = 3.5

x3 = np.zeros(N)
for k in range(N):
    if np.all(np.zeros(2) <= pp[k,:]) and np.all(pp[k,:] <= np.ones(2)):
        x3[k] = u(dl.Point(pp[k,:]))
    else:
        x3[k] = exterior_fill_value

B.exterior_fill_value = exterior_fill_value

x4 = B * u.vector()[:]
x4[outside_mesh_mask] = exterior_fill_value

err_exterior_fill = np.linalg.norm(x4 - x3)
print('err_exterior_fill=', err_exterior_fill)

err_exterior_fill= 8.460899244930976e-16


In [5]:
q = np.random.randn(V.dim())
z = np.random.randn(pp.shape[0])

err_transpose = np.abs(np.dot(B * q, z) - np.dot(q, B.T * z))
print('err_transpose=', err_transpose)

err_transpose= 1.7763568394002505e-15


In [37]:
B2, nonzero_cols = pointwise_observation_matrix(pp, V, nonzero_columns_only=True)

u_vec = u.vector()[:]

t = time()
x5 = B * u_vec
dt_allcols = time() - t
print('dt_allcols=', dt_allcols)

uvc = u_vec[nonzero_cols]
t = time()
x6 = B2 * uvc
dt_nonzero_cols = time() - t
print('dt_nonzero_cols=', dt_nonzero_cols)

err_nonzero_cols = np.linalg.norm(x6 - x5)
print('err_nonzero_cols=', err_nonzero_cols)


dt_allcols= 0.00011873245239257812
dt_nonzero_cols= 7.271766662597656e-05
err_nonzero_cols= 0.0
